In [1]:
import pandas as pd
import struct
import re
import numpy as np

import matplotlib.colors as cols
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# %matplotlib inline

## Import .pos

In [2]:
def read_pos(f):
    # read in the data
    n = len(file(f).read())/4
    d = struct.unpack('>'+'f'*n,file(f).read(4*n)) 
                    # '>' denotes 'big-endian' byte order
    # unpack data
    pos = pd.DataFrame({'x': d[0::4],
                        'y': d[1::4],
                        'z': d[2::4],
                        'Da': d[3::4]})
    return pos

In [3]:
pos = read_pos('R31_04588-v02.pos')

## Import .epos

In [4]:
def read_epos(f):
    # read in the data
    n = len(file(f).read())/4
    rs = n / 11
    d = struct.unpack('>'+'fffffffffII'*rs,file(f).read(4*n)) 
                    # '>' denotes 'big-endian' byte order
    # unpack data
    pos = pd.DataFrame({'x': d[0::11],
                        'y': d[1::11],
                        'z': d[2::11],
                        'Da': d[3::11],
                        'ns': d[4::11],
                        'DC_kV': d[5::11],
                        'pulse_kV': d[6::11],
                        'det_x': d[7::11],
                        'det_y': d[8::11],
                        'pslep': d[9::11], # pulses since last event pulse
                        'ipp': d[10::11]}) # ions per pulse
    return pos

In [5]:
epos = read_epos('R31_04588-v02.epos')

## Import .rrng

In [6]:
def read_rrng(f):
    rf = open(f,'r').readlines()

    patterns = re.compile(r'Ion([0-9]+)=([A-Za-z0-9]+).*|Range([0-9]+)=(\d+.\d+) +(\d+.\d+) +Vol:(\d+.\d+) +([A-Za-z:0-9 ]+) +Color:([A-Z0-9]{6})')

    ions = []
    rrngs = []
    for line in rf:
        m = patterns.search(line)
        if m:
            if m.groups()[0] is not None:
                ions.append(m.groups()[:2])
            else:
                rrngs.append(m.groups()[2:])

    ions = pd.DataFrame(ions, columns=['number','name'])
    ions.set_index('number',inplace=True)
    rrngs = pd.DataFrame(rrngs, columns=['number','lower','upper','vol','comp','colour'])
    rrngs.set_index('number',inplace=True)
    
    rrngs[['lower','upper','vol']] = rrngs[['lower','upper','vol']].astype(float)
    rrngs[['comp','colour']] = rrngs[['comp','colour']].astype(str)
    
    return ions,rrngs

In [7]:
ions, rrngs = read_rrng('4588_whole_tip_figure.rrng')

## Assign ion labels

In [8]:
def label_ions(pos,rrngs):
    pos['comp'] = ''
    pos['colour'] = '#FFFFFF'
    
    for n,r in rrngs.iterrows():
        pos.loc[(pos.Da >= r.lower) & (pos.Da <= r.upper),['comp','colour']] = [r['comp'],'#' + r['colour']]
    
    return pos

In [9]:
lpos = label_ions(pos,rrngs)

## Deconvolve Complex Ions

In [15]:
def deconvolve(lpos):
    """Takes a composition-labelled pos file, and deconvolves
    the complex ions. Produces a dataframe of the same input format
    with the extra columns:
       'element': element name
       'n': stoichiometry
    For complex ions, the location of the different components is not
    altered - i.e. xyz position will be the same for several elements."""
      
    out = []
    pattern = re.compile(r'([A-Za-z]+):([0-9]+)')

    for g,d in lpos.groupby('comp'):
        if g is not '':
            for i in range(len(g.split(' '))):
                tmp = d.copy()
                cn = pattern.search(g.split(' ')[i]).groups()
                tmp['element'] = cn[0]
                tmp['n'] = cn[1]
                out.append(tmp.copy())
    return pd.concat(out)

In [16]:
dpos = deconvolve(lpos)

## Visualise Volume

In [78]:
from vispy import app,scene,mpl_plot
canvas = scene.SceneCanvas(keys='interactive')
view = canvas.central_widget.add_view()
leg = scene.widgets.ViewBox(parent=view, border_color='red', bgcolor='k')

mpl_plot.scatter()

TypeError: scatter() takes at least 2 arguments (0 given)

In [181]:
def volvis(pos, size=2, alpha=1):
    """Displays a 3D point cloud in an OpenGL viewer window.
    If points are not labelled with colours, point brightness
    is determined by Da values (higher = whiter)"""
    from vispy import app,scene,mpl_plot
    import numpy as np
    import sys
    
    canvas = scene.SceneCanvas('APT Volume',keys='interactive')
    view = canvas.central_widget.add_view()
    view.camera = scene.TurntableCamera(up='z')
    
    cpos = pos.loc[:,['x','y','z']].values
    if 'colour' in pos.columns:
        colours = np.asarray(list(pos.colour.apply(cols.hex2color)))
    else:
        Dapc = lpos.Da.values / lpos.Da.max()
        colours = np.array(zip(Dapc,Dapc,Dapc))
    if alpha is not 1:
        np.hstack([colours, np.array([0.5] * len(colours))[...,None]])
    
    p1 = scene.visuals.Markers()
    p1.set_data(cpos, face_color=colours, edge_width=0, size=size)

    view.add(p1)
    
    # make legend
    ions = []
    cs = []
    for g,d in pos.groupby('colour'):
        ions.append(re.sub(r':1?|\s?','',d.comp.iloc[0]))
        cs.append(cols.hex2color(g))
    ions = np.array(ions)
    cs = np.asarray(cs)

    pts = np.array([[20] * len(ions), np.linspace(20,20*len(ions), len(ions))]).T
    tpts = np.array([[30] * len(ions), np.linspace(20,20*len(ions), len(ions))]).T
    
    legb = scene.widgets.ViewBox(parent=view, border_color='red', bgcolor='k')
    legb.pos = 0,0
    legb.size = 100,20*len(ions)+20
    
    leg = scene.visuals.Markers()
    leg.set_data(pts, face_color=cs)
    legb.add(leg)
    
    legt = scene.visuals.Text(text=ions,pos=tpts,color='white', anchor_x='left', anchor_y='center', font_size=10)
    
    legb.add(legt)
    
    # show viewer
    canvas.show()
    if sys.flags.interactive == 0: 
        app.run()

In [177]:
volvis(dpos,alpha=0.6)

In [182]:
volvis(dpos.loc[dpos.element=='Na',:])